Один из вариантов решения нетривиальных задач - декомпозиция. Применение искуственных нейронных сетей (ИНС), по своей сути также, является сложной задачей, которую можно разбить на подзадачи.

Стоит отметить, что при создании проектов, основанных на использовании ИНС, можно выделить такие блоки, изменять которые возможно независимо от остальных. Данный факт можно воспринимать как возможность создавать свои проекты из отдельных частей, как конструктор.

Мы предлагаем разделить процесс простроения проектов, связанных с обучением ИНС на следующие части:
- **Работа с набором изначальных данных**. Анализ изначальных данных и осознание их природы, выявление потенциальных проблем.
- **Блок загрузки данных в ИНС**. Включает в себя всевозможные преобразования данных, выполняемые с целью улучшения качества обучения ИНС, и преобразования, необходимые для представления данных в численном виде.
- **Архитектура используемой ИНС**. Подразумевает проектирование или подбор архитектуры ИНС, которую можно использовать для решения необходимой задачи.
- **Выбор функции потерь, метрик, оптимизатора, используемых для обучения**. Основываясь на задаче и возможных проблемах необходимо выбрать используемые
функции и алгоритмы.
- **Составление цикла обучения ИНС**. В этом блоке необходимо выбрать оптимальный способ обучения ИНС, способ подсчёта метрик и критерии остановки.
- **Оценка итогового качетсва ИНС**. Проверка корректности работы ИНС различными способами.
- **Составления приложения для использования ИНС**. Состоит из разработки итогового приложения, которое будет использоваться для решения задачи.

Дополнительно возможно отметить другие блоки, связанные с оформлением кода:
- **Подключение библиотек.**
- **Определение гиперпараметров, констант, классов, функций.**
- **Ячейки визуализации данных и вывода информации о данных.**

Стоит отметить, что выше представлен один из многих вариантов разбиения. Описание пунктов является кратким, и его, разумеется, можно расширить. В простых проектах некоторые блоки могут отсутсвовать, хотя, на самом деле, их использование является необходимым.

Ниже представлен код, в котором выполняется обучение ИНС решению задачи классификации одежды (набор данных FashionMNIST, используемая библиотека - PyTorch). Он разбит на 2 ячкейки.

**Предлагается выполнить декомпозицию кода и разместить его фрагменты в различные ячейки**. Также, если Вы захотите, возможно используя MarkDown (Текст) подписать ячейки кода и информацию, которую Вы считаете важной. Гайд по оформлению: https://youtu.be/LLFOZ2vN7ss.

Возможно использование разбиения кода на блоки другим образом.

Ссылка на оригинальную статью: https://pytorch.org/tutorials/beginner/introyt/trainingyt.html. **Если Вы понимаете код на Python и умеете работать с документацией библиотеки, мы рекомендуем самостоятельно разбить код, после чего рассмотреть статью**. Если Вы пока не дошли до этого уровня (этот уровень уже можно назвать продвинутым), то можете изучить статью и сформулировать по ней вопросы.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np

import torch.nn as nn
import torch.nn.functional as F


transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

training_set = torchvision.datasets.FashionMNIST('./data', train=True, transform=transform, download=True)
validation_set = torchvision.datasets.FashionMNIST('./data', train=False, transform=transform, download=True)

training_loader = torch.utils.data.DataLoader(training_set, batch_size=4, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=4, shuffle=False)

classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
        'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

print('Training set has {} instances'.format(len(training_set)))
print('Validation set has {} instances'.format(len(validation_set)))

def matplotlib_imshow(img, one_channel=False):
    if one_channel:
        img = img.mean(dim=0)
    img = img / 2 + 0.5
    npimg = img.numpy()
    if one_channel:
        plt.imshow(npimg, cmap="Greys")
    else:
        plt.imshow(np.transpose(npimg, (1, 2, 0)))

dataiter = iter(training_loader)
images, labels = next(dataiter)

img_grid = torchvision.utils.make_grid(images)
matplotlib_imshow(img_grid, one_channel=True)
print('  '.join(classes[labels[j]] for j in range(4)))

KeyboardInterrupt: 

In [ ]:
class GarmentClassifier(nn.Module):
    def __init__(self):
        super(GarmentClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = GarmentClassifier()

loss_fn = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    for i, data in enumerate(training_loader):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = model(inputs)

        loss = loss_fn(outputs, labels)
        loss.backward()

        optimizer.step()

        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(training_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss


timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
epoch_number = 0

EPOCHS = 5

best_vloss = 1_000_000.

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch_number + 1))

    model.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)


    running_vloss = 0.0
    model.eval()

    with torch.no_grad():
        for i, vdata in enumerate(validation_loader):
            vinputs, vlabels = vdata
            voutputs = model(vinputs)
            vloss = loss_fn(voutputs, vlabels)
            running_vloss += vloss

    avg_vloss = running_vloss / (i + 1)
    print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()

    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'model_{}_{}'.format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)

    epoch_number += 1